## **Three dims reconstruction of 3D point cloud**

    This notebook demonstrate:
    1.3D point cloud reconstruction of the whole body;
    2.3D point cloud reconstruction of the tissues;
    3.Orthogonal slicing of 3D point cloud data;
    4.Axial slicing of 3D point cloud data.

### **Packages**

In [1]:
import anndata as ad
import stDrosophila as sd

/home/yao/anaconda3/envs/stDrosophila-release/lib/python3.7/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### **Example data (Drosophila E16-18-a)**

In [2]:
file = r"/home/yao/BGIpy37_pytorch113/drosophila_E16_18/E16-18_a_SCT_anno.h5ad"
adata = ad.read(file)
adata.obs["x"] = adata.obs["x"] - adata.obs["x"].min()
adata.obs["y"] = adata.obs["y"] - adata.obs["y"].min()
adata.obs["z"] = adata.obs["z"] - 4.9
adata.obs[["x", "y", "z"]] = adata.obs[["x", "y", "z"]].round(2)
adata.obsm["spatial"] = adata.obs[["x", "y", "z"]].values
print(adata)
print(adata.obs["anno"].unique())

AnnData object with n_obs × n_vars = 14634 × 518
    obs: 'slice', 'x', 'y', 'z', 'anno'
    obsm: 'raw_spatial', 'spatial'
    layers: 'raw_counts'
['salivary gland', 'epidermis', 'CNS', 'carcass', 'fat body', 'muscle', 'trachea', 'midgut', 'hemolymph', 'foregut']
Categories (10, object): ['CNS', 'carcass', 'epidermis', 'fat body', ..., 'midgut', 'muscle', 'salivary gland', 'trachea']


### **3D reconstrcution of the whole body**

3D point cloud can be reconstructed by this method.

In [10]:
pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="rainbow", alphamap=1.0)
sd.pl.three_d_plot(mesh=pcd, key="groups", legend_size=(0.2, 0.2), jupyter=True)

### **3D reconstrcution of the tissue**

If you only want to display one of the tissues, you can use the following two methods (here takes the visualization of the fat body as an example):

1.Use the mask parameter to set the tissues that do not need to be displayed as mask.

In [9]:
mask = ['salivary gland', 'epidermis', 'CNS', 'carcass', 'muscle', 'trachea', 'midgut', 'hemolymph', 'foregut']
fb_pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", mask=mask, colormap="skyblue", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

2.Modify the original adata so that only one tissue remains in the `groupby`.

In [11]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="skyblue", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

### **Orthogonal slicing**

### **Axial slicing**